In [104]:
# 3D adaptation of ResNet50, based on the 2D version of : https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py
# THE MODEL CAN BE CONSTRUCT BUT NOT FIT YET
from scipy import interpolate
from skimage import io
from keras.layers import Input, Add, Dense, Activation, ZeroPadding3D, BatchNormalization, Flatten, Conv3D, AveragePooling3D, MaxPooling3D
from keras.models import Model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model, to_categorical
from keras.initializers import glorot_uniform
import numpy as np
import deepcell
import keras.backend as K
K.set_learning_phase(1)

In [105]:
def interpolation_z_axis(data, multiplicator):
    result_img = np.empty((data.shape[0], data.shape[1] * multiplicator, data.shape[2], data.shape[3]))

    for i in range(data.shape[0]):
        print(i)
        X = data[i]
        temp_image = np.empty((X.shape[0] * multiplicator, X.shape[1], X.shape[2]))
        temp_image[:] = np.nan
        for j in range(X.shape[0]):
            temp_image[j * multiplicator] = X[j]
        temp_image[-1] = temp_image[-2]
        indexes = np.arange(temp_image.shape[0])
        good = np.isfinite(temp_image).all(axis=(1, 2))
        f = interpolate.interp1d(indexes[good], temp_image[good], bounds_error=False, axis=0)
        B = f(indexes)
        result_img[i] = B

    #io.imsave('InterpolatedX_train_' + str(multiplicator) + '.tiff', result_img.astype('uint8'), 'tifffile')
    print("Final shape : ", result_img.shape)
    return result_img.astype('uint8')

In [106]:
def identity_block(X, f, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value.
    X_shortcut = X

    # First component of main path
    X = Conv3D(filters=F1, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='valid', name=conv_name_base + '2a',
               kernel_initializer=glorot_uniform(seed=0), data_format = 'channels_last')(X)
    X = BatchNormalization(axis=1, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv3D(filters=F2, kernel_size=(f, f, f), strides=(1, 1, 1), padding='same', name=conv_name_base + '2b',
               kernel_initializer=glorot_uniform(seed=0), data_format = 'channels_last')(X)
    X = BatchNormalization(axis=1, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv3D(filters=F3, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='valid', name=conv_name_base + '2c',
               kernel_initializer=glorot_uniform(seed=0), data_format = 'channels_last')(X)
    X = BatchNormalization(axis=1, name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [107]:
def convolutional_block(X, f, filters, stage, block, s=2):
    """conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path
    X = Conv3D(F1, (1, 1, 1), strides=(s, s, s), name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0), data_format = 'channels_last')(X)
    X = BatchNormalization(axis=1, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv3D(filters=F2, kernel_size=(f, f, f), strides=(1, 1, 1), padding='same', name=conv_name_base + '2b',
               kernel_initializer=glorot_uniform(seed=0), data_format = 'channels_last')(X)
    X = BatchNormalization(axis=1, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv3D(filters=F3, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='valid', name=conv_name_base + '2c',
               kernel_initializer=glorot_uniform(seed=0), data_format = 'channels_last')(X)
    X = BatchNormalization(axis=1, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH ####
    X_shortcut = Conv3D(filters=F3, kernel_size=(1, 1, 1), strides=(s, s, s), padding='valid', name=conv_name_base + '1',
                        kernel_initializer=glorot_uniform(seed=0), data_format = 'channels_last')(X_shortcut)
    X_shortcut = BatchNormalization(axis=1, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [116]:

def ResNet50(input_shape, classes=6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    print("Input shape : ", X_input.shape)

    # Zero-Padding
    X = ZeroPadding3D(padding=(3, 3, 3), data_format = 'channels_last')(X_input)
    print("X shape Zero Padding : ", X.shape)

    # Stage 1
    X = Conv3D(64, (7, 7, 7), strides=(2, 2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0), data_format = 'channels_last')(X)
    print("X shape conv3D: ", X.shape)

    X = BatchNormalization(axis=1, name='bn_conv1')(X)
    print("X shape BatchNormalization: ", X.shape)

    X = Activation('relu')(X)
    print("X shape Activation: ", X.shape)

    X = MaxPooling3D((3, 3, 3), strides=(1, 1, 1))(X)
    print("X shape Stage 1 : ", X.shape)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 64], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 64], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 64], stage=2, block='c')

    print("X shape Stage 2 : ", X.shape)

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 128], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 128], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 128], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 128], stage=3, block='d')

    print("X shape Stage 3 : ", X.shape)

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 256], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 256], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 256], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 256], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 256], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 256], stage=4, block='f')

    print("X shape Stage 4 : ", X.shape)

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 512], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 512], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 512], stage=5, block='c')

    print("X shape Stage 5 : ", X.shape)

    # AVGPOOL
    X = AveragePooling3D((7, 7, 7), name="avg_pool")(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    print("Output shape : ", X.shape)

    return model


In [109]:
# Download the data (saves to ~/.keras/datasets)
filename = 'mousebrain.npz'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.mousebrain.load_data(filename)
X_train_2 =  X_train[:,:,:,:,0]
y_train_2 = y_train[:,:,:,:,0]
print('X_train_2.shape: {}\ny.shape: {}'.format(X_train_2.shape, y_train.shape))

X_train_2.shape: (176, 15, 256, 256)
y.shape: (176, 15, 256, 256, 1)


In [110]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(176, 15, 256, 256, 1) (176, 15, 256, 256, 1)
(44, 15, 256, 256, 1) (44, 15, 256, 256, 1)


In [117]:
model = ResNet50(input_shape=(120,256,256,1), classes=2)

Input shape :  (?, 120, 256, 256, 1)
X shape Zero Padding :  (?, 126, 262, 262, 1)
X shape conv3D:  (?, 60, 128, 128, 64)
X shape BatchNormalization:  (?, 60, 128, 128, 64)
X shape Activation:  (?, 60, 128, 128, 64)
X shape Stage 1 :  (?, 58, 126, 126, 64)
X shape Stage 2 :  (?, 58, 126, 126, 64)
X shape Stage 3 :  (?, 29, 63, 63, 128)
X shape Stage 4 :  (?, 15, 32, 32, 256)
X shape Stage 5 :  (?, 8, 16, 16, 512)
Output shape :  (?, 2)


In [118]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
X_train_interpol = interpolation_z_axis(X_train[:,:,:,:,0], 8)
y_train_interpol = interpolation_z_axis(y_train[:,:,:,:,0] ,8)

X_train_interpol = X_train_interpol[:,:,:,:, np.newaxis]
y_train_interpol = y_train_interpol[:,:,:,:, np.newaxis]
print('X train interpol : "', X_train_interpol.shape)
print('X train interpol : "', y_train_interpol.shape)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
Final shape :  (176, 120, 256, 256)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


In [113]:
print('X train interpol : "', X_train_interpol.shape)
print('Y train interpol : "', y_train_interpol.shape)

X train interpol : " (176, 120, 256, 256, 1)
Y train interpol : " (176, 120, 256, 256, 1)


In [114]:
model.input_shape

(None, 120, 256, 256, 1)

In [115]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 120, 256, 256 0                                            
__________________________________________________________________________________________________
zero_padding3d_13 (ZeroPadding3 (None, 126, 262, 262 0           input_13[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv3D)                  (None, 60, 128, 128, 22016       zero_padding3d_13[0][0]          
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 60, 128, 128, 240         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [123]:
model.fit(X_train_interpol, one_hot_labels, epochs=25, batch_size=32)

ValueError: Error when checking target: expected fc2 to have 2 dimensions, but got array with shape (176, 15, 256, 256, 100)

In [91]:
model.layers[-1].name

'fc6'

In [44]:
y_train_tc = to_categorical(y_train_interpol, num_classes=6)

IndexError: index 8 is out of bounds for axis 1 with size 6

In [69]:
y_train_tc = np.reshape(y_train_interpol, (346030080, 4))

In [74]:
y_train_tc = to_categorical(y_train_interpol, 6) 

IndexError: index 8 is out of bounds for axis 1 with size 6

In [72]:
from keras.utils.np_utils import to_categorical

In [78]:
dataset = tf.data.Dataset.from_tensor_slices((X_train_interpol,y_train_interpol))

In [77]:
import tensorflow as tf

In [80]:
dataset

<TensorSliceDataset shapes: ((120, 256, 256, 1), (120, 256, 256, 1)), types: (tf.uint8, tf.uint8)>

In [82]:
model.fit(dataset.make_one_shot_iterator(), epochs=25, batch_size=32)

InvalidArgumentError: Cannot serialize protocol buffer of type tensorflow.FunctionDef as the serialized size (2768241222bytes) would be larger than the limit (2147483647 bytes)

In [125]:
one_hot_labels = to_categorical(y_train_interpol, num_classes = 20)

MemoryError: 

array([[[[[1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          ...,
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.]],

         [[1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          ...,
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.]],

         [[1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          ...,
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.]],

         ...,

         [[1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0.

In [126]:
import tensorflow as tf
tf.keras.backend.clear_session()